# Fine-tuning with Contrastive Loss

Learn to use contrastive loss (InfoNCE) for learning discriminative embeddings.

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

## What is Contrastive Loss?

Contrastive loss (also called InfoNCE or NT-Xent) learns by:
- Pulling similar pairs together
- Pushing dissimilar pairs apart

**Key difference from Triplet Loss:**
- Triplet: Uses explicit anchor-positive-negative
- Contrastive: Uses pairs with similarity labels (0 or 1)

```
Pair 1: ["How to reset password?", "Click forgot password", label=1.0] ← Similar
Pair 2: ["How to reset password?", "Pricing starts at $29", label=0.0] ← Dissimilar
```

## Load Training Data

In [ ]:
# Load hard pairs from extended dataset (500 examples)
with open('../fixtures/input/training_pairs_hard.json', 'r') as f:
    pairs_data = json.load(f)

print(f"Loaded {len(pairs_data)} hard training examples\n")

# Show example
example = pairs_data[0]
print("Training example structure:")
print(f"  Query: {example['query']}")
print(f"  Positive: {example['positive']}")
print(f"  Negative: {example['negative']}")

print("\n💡 Hard negatives are semantically similar but not relevant")
print("   This makes the model learn fine-grained distinctions!")

## Prepare Contrastive Pairs

Create both positive pairs (label=1.0) and negative pairs (label=0.0).

In [ ]:
# Create InputExample objects with labels
train_examples = []

for item in pairs_data:
    # Positive pair (label = 1.0)
    train_examples.append(
        InputExample(texts=[item['query'], item['positive']], label=1.0)
    )
    
    # Negative pair (label = 0.0)
    train_examples.append(
        InputExample(texts=[item['query'], item['negative']], label=0.0)
    )

print(f"Created {len(train_examples)} training pairs")
print(f"  Positive pairs: {len(train_examples) // 2}")
print(f"  Negative pairs: {len(train_examples) // 2}")
print(f"\nExample format:")
print(f"  Texts: {train_examples[0].texts}")
print(f"  Label: {train_examples[0].label}")

## Split Train/Validation

In [ ]:
# Split 80/20
split_idx = int(0.8 * len(train_examples))
train_data = train_examples[:split_idx]
val_data = train_examples[split_idx:]

print(f"Training pairs: {len(train_data)}")
print(f"Validation pairs: {len(val_data)}")

## Create DataLoader

In [ ]:
train_dataloader = DataLoader(
    train_data,
    shuffle=True,
    batch_size=16
)

print(f"DataLoader created:")
print(f"  Batch size: 16")
print(f"  Number of batches: {len(train_dataloader)}")

## Configure Contrastive Loss (OnlineContrastiveLoss)

**Parameters:**
- `distance_metric`: Cosine or Euclidean
- `margin`: Minimum distance for negative pairs
- `size_average`: Average loss over batch

In [ ]:
# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Define contrastive loss
train_loss = losses.OnlineContrastiveLoss(
    model=model,
    distance_metric=losses.SiameseDistanceMetric.COSINE_DISTANCE,
    margin=0.5  # Negative pairs should be at least 0.5 apart
)

print("✓ Loss function: OnlineContrastiveLoss")
print("  Distance metric: Cosine")
print("  Margin: 0.5")

## Setup Evaluator

In [ ]:
# Prepare validation data
val_sentences1 = [ex.texts[0] for ex in val_data]
val_sentences2 = [ex.texts[1] for ex in val_data]
val_scores = [ex.label for ex in val_data]

evaluator = EmbeddingSimilarityEvaluator(
    sentences1=val_sentences1,
    sentences2=val_sentences2,
    scores=val_scores,
    name='contrastive_eval'
)

print(f"✓ Evaluator created with {len(val_data)} validation pairs")

## Fine-tune with Contrastive Loss

In [ ]:
# Training configuration
num_epochs = 3  # Reduced for 500 examples - more data needs fewer epochs
warmup_steps = int(0.1 * len(train_dataloader) * num_epochs)
eval_steps = max(50, len(train_dataloader) // 4)  # Evaluate 4 times per epoch

print(f"Training configuration:")
print(f"  Total examples: {len(train_data)} ({len(train_data)//2} positive + {len(train_data)//2} negative)")
print(f"  Hard negatives: ✓ Challenging examples")
print(f"  Epochs: {num_epochs}")
print(f"  Warmup steps: {warmup_steps}")
print(f"  Evaluation frequency: every {eval_steps} steps")
print(f"  Total steps: {len(train_dataloader) * num_epochs}")

# Evaluate before training
print("\n" + "="*60)
print("INITIAL EVALUATION (before training)")
print("="*60)
initial_result = evaluator(model, output_path='../output/contrastive_finetuned_model')
# EmbeddingSimilarityEvaluator returns dict like {'contrastive_eval_cosine_spearman': 0.82}
if isinstance(initial_result, dict):
    # Get spearman correlation value
    if 'cosine_spearman' in str(initial_result):
        initial_score = [v for k, v in initial_result.items() if 'spearman' in k][0]
    else:
        initial_score = list(initial_result.values())[0]
else:
    initial_score = initial_result
print(f"Initial Spearman Correlation: {initial_score:.4f}")
print(f"  (1.0 = perfect, 0.0 = no correlation, -1.0 = inverse)")
print("="*60 + "\n")

# Fine-tune
print("Starting fine-tuning with contrastive loss on HARD examples...")
print("Watch for evaluation outputs below:\n")

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    evaluator=evaluator,
    evaluation_steps=eval_steps,  # Evaluate more frequently
    output_path='../output/contrastive_finetuned_model',
    show_progress_bar=True,
    save_best_model=True,  # Only save best model
    use_amp=False,  # Disable mixed precision
    checkpoint_save_steps=10000,  # Don't save intermediate checkpoints
    checkpoint_save_total_limit=1,  # Keep only best model
    optimizer_params={'lr': 2e-5}  # Learning rate for better convergence
)

print("\n" + "="*60)
print("✓ Fine-tuning complete!")
print("="*60)

# Load and display metrics
import pandas as pd
import os

# NOTE: sentence-transformers saves evaluation results to 'eval/' subfolder
# with prefix from evaluator class name (e.g., 'similarity_evaluation_')
metrics_file = '../output/contrastive_finetuned_model/eval/similarity_evaluation_contrastive_eval_results.csv'
if os.path.exists(metrics_file):
    print("\n" + "="*60)
    print("TRAINING METRICS SUMMARY")
    print("="*60)
    metrics_df = pd.read_csv(metrics_file)
    
    # Filter out pre-training evaluation (epoch=-1, steps=-1)
    metrics_df = metrics_df[(metrics_df['epoch'] >= 0) & (metrics_df['steps'] >= 0)]
    
    # Show first and last few rows
    print("\nFirst evaluations:")
    print(metrics_df.head(3).to_string(index=False))
    if len(metrics_df) > 6:
        print("\n...")
        print("\nFinal evaluations:")
        print(metrics_df.tail(3).to_string(index=False))
    
    print("\n" + "="*60)
    # EmbeddingSimilarityEvaluator tracks multiple metrics
    if 'cosine_spearman' in metrics_df.columns:
        metric_col = 'cosine_spearman'
    elif 'spearman' in metrics_df.columns:
        metric_col = 'spearman'
    else:
        metric_col = metrics_df.columns[-1]
    
    best_score = metrics_df[metric_col].max()
    best_step = metrics_df.loc[metrics_df[metric_col].idxmax(), 'steps']
    improvement = best_score - initial_score
    improvement_pct = (improvement / abs(initial_score) * 100) if initial_score != 0 else 0
    
    print(f"📊 RESULTS:")
    print(f"  Initial Score:  {initial_score:.4f}")
    print(f"  Best Score:     {best_score:.4f} (at step {int(best_step)})")
    print(f"  Improvement:    +{improvement:.4f} ({improvement_pct:+.1f}%)")
    print("="*60)
    
    # Note about training loss
    print("\n💡 NOTE: 'No log' in Training/Validation Loss columns is NORMAL.")
    print("   sentence-transformers logs evaluation metrics (spearman)")
    print("   but doesn't log raw loss values in the progress bar.")
    print("   The evaluation metrics are saved to CSV and shown above.")
    
    if improvement > 0.1:
        print("\n✅ Model successfully improved on hard negatives!")
    elif improvement > 0:
        print("\n⚠️  Small improvement - hard negatives are challenging")
    else:
        print("\n⚠️  No improvement - may need more epochs or different hyperparameters")
else:
    print("\n⚠️  Metrics file not found at:")
    print(f"   {metrics_file}")
    print("   Check that the evaluator and output_path are correctly configured")

## Visualize Training Progress

In [ ]:
# Plot training progress
import pandas as pd
import os

metrics_file = '../output/contrastive_finetuned_model/eval/similarity_evaluation_contrastive_eval_results.csv'
if os.path.exists(metrics_file):
    metrics_df = pd.read_csv(metrics_file)
    
    # Filter out pre-training evaluation (epoch=-1, steps=-1)
    metrics_df = metrics_df[(metrics_df['epoch'] >= 0) & (metrics_df['steps'] >= 0)]
    
    # Determine which metric column to use
    metric_col = None
    if 'cosine_spearman' in metrics_df.columns:
        metric_col = 'cosine_spearman'
        metric_name = 'Spearman Correlation (Cosine)'
    elif 'spearman' in metrics_df.columns:
        metric_col = 'spearman'
        metric_name = 'Spearman Correlation'
    
    if metric_col:
        fig, ax = plt.subplots(figsize=(12, 5))
        
        ax.plot(metrics_df['steps'], metrics_df[metric_col], marker='o', linewidth=2, markersize=6)
        ax.axhline(initial_score, color='red', linestyle='--', label=f'Initial: {initial_score:.4f}')
        ax.set_xlabel('Training Steps', fontsize=12)
        ax.set_ylabel(metric_name, fontsize=12)
        ax.set_title('Contrastive Loss Training Progress', fontsize=14, fontweight='bold')
        ax.grid(True, alpha=0.3)
        ax.legend()
        
        # Annotate best score
        best_idx = metrics_df[metric_col].idxmax()
        best_step = metrics_df.loc[best_idx, 'steps']
        best_score = metrics_df.loc[best_idx, metric_col]
        ax.annotate(f'Best: {best_score:.4f}', 
                    xy=(best_step, best_score),
                    xytext=(10, -20),
                    textcoords='offset points',
                    bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.7),
                    arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))
        
        plt.tight_layout()
        plt.show()
        
        print(f"\n{metric_name} improved from {initial_score:.4f} to {best_score:.4f}")
        print(f"Total improvement: +{(best_score - initial_score):.4f} ({((best_score - initial_score) / abs(initial_score) * 100):.1f}%)")
    else:
        print("Metric columns found:", metrics_df.columns.tolist())
else:
    print("⚠️  No metrics file found. Training may not have completed or evaluator didn't run.")
    print(f"   Expected path: {metrics_file}")

## Compare Generic vs Fine-tuned

In [ ]:
# Load models
generic_model = SentenceTransformer('all-MiniLM-L6-v2')
finetuned_model = SentenceTransformer('../output/contrastive_finetuned_model')

# Test on validation examples
test_examples = val_data[:20]

generic_correct = 0
finetuned_correct = 0

for ex in test_examples:
    # Generic
    emb1 = generic_model.encode(ex.texts[0])
    emb2 = generic_model.encode(ex.texts[1])
    gen_sim = cosine_similarity([emb1], [emb2])[0][0]
    
    # Fine-tuned
    emb1 = finetuned_model.encode(ex.texts[0])
    emb2 = finetuned_model.encode(ex.texts[1])
    ft_sim = cosine_similarity([emb1], [emb2])[0][0]
    
    # Check if prediction matches label
    # Positive pairs (label=1.0) should have similarity > 0.5
    # Negative pairs (label=0.0) should have similarity < 0.5
    threshold = 0.5
    
    if ex.label == 1.0:
        if gen_sim > threshold:
            generic_correct += 1
        if ft_sim > threshold:
            finetuned_correct += 1
    else:
        if gen_sim < threshold:
            generic_correct += 1
        if ft_sim < threshold:
            finetuned_correct += 1

print(f"Classification accuracy (threshold=0.5):")
print(f"  Generic model: {generic_correct}/{len(test_examples)} = {100*generic_correct/len(test_examples):.1f}%")
print(f"  Fine-tuned model: {finetuned_correct}/{len(test_examples)} = {100*finetuned_correct/len(test_examples):.1f}%")
print(f"\n✓ Improvement: +{finetuned_correct - generic_correct} correct predictions")

## Visualize Similarity Distributions

In [ ]:
# Collect similarities for positive and negative pairs
gen_pos_sims, gen_neg_sims = [], []
ft_pos_sims, ft_neg_sims = [], []

for ex in val_data[:50]:
    # Generic
    emb1 = generic_model.encode(ex.texts[0])
    emb2 = generic_model.encode(ex.texts[1])
    gen_sim = cosine_similarity([emb1], [emb2])[0][0]
    
    # Fine-tuned
    emb1 = finetuned_model.encode(ex.texts[0])
    emb2 = finetuned_model.encode(ex.texts[1])
    ft_sim = cosine_similarity([emb1], [emb2])[0][0]
    
    if ex.label == 1.0:
        gen_pos_sims.append(gen_sim)
        ft_pos_sims.append(ft_sim)
    else:
        gen_neg_sims.append(gen_sim)
        ft_neg_sims.append(ft_sim)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Generic model
axes[0].hist(gen_pos_sims, alpha=0.6, label='Positive pairs', bins=15, color='green')
axes[0].hist(gen_neg_sims, alpha=0.6, label='Negative pairs', bins=15, color='red')
axes[0].axvline(0.5, color='black', linestyle='--', label='Threshold')
axes[0].set_xlabel('Similarity')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Generic Model')
axes[0].legend()

# Fine-tuned model
axes[1].hist(ft_pos_sims, alpha=0.6, label='Positive pairs', bins=15, color='green')
axes[1].hist(ft_neg_sims, alpha=0.6, label='Negative pairs', bins=15, color='red')
axes[1].axvline(0.5, color='black', linestyle='--', label='Threshold')
axes[1].set_xlabel('Similarity')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Fine-tuned Model (Contrastive Loss)')
axes[1].legend()

plt.tight_layout()
plt.show()

print("\n📊 Better separation = Less overlap between green and red")

## Statistics

In [ ]:
print("Similarity Statistics:\n")

print("Generic Model:")
print(f"  Positive pairs: μ={np.mean(gen_pos_sims):.3f}, σ={np.std(gen_pos_sims):.3f}")
print(f"  Negative pairs: μ={np.mean(gen_neg_sims):.3f}, σ={np.std(gen_neg_sims):.3f}")
print(f"  Separation: {np.mean(gen_pos_sims) - np.mean(gen_neg_sims):.3f}")

print("\nFine-tuned Model:")
print(f"  Positive pairs: μ={np.mean(ft_pos_sims):.3f}, σ={np.std(ft_pos_sims):.3f}")
print(f"  Negative pairs: μ={np.mean(ft_neg_sims):.3f}, σ={np.std(ft_neg_sims):.3f}")
print(f"  Separation: {np.mean(ft_pos_sims) - np.mean(ft_neg_sims):.3f}")

improvement = (
    (np.mean(ft_pos_sims) - np.mean(ft_neg_sims)) - 
    (np.mean(gen_pos_sims) - np.mean(gen_neg_sims))
)
print(f"\n✓ Separation improvement: +{improvement:.3f}")

## Test on Real Queries

In [ ]:
test_queries = [
    {
        "query": "forgot my login credentials",
        "docs": [
            "Reset password from account settings page",
            "Contact support for billing questions",
            "API documentation available online"
        ]
    },
    {
        "query": "what does BERT do?",
        "docs": [
            "BERT is bidirectional transformer for NLP",
            "Database backup runs nightly at midnight",
            "Professional plan includes 25 users"
        ]
    }
]

print("Query Ranking Test:\n")

for test in test_queries:
    print(f"Query: {test['query']}")
    
    # Generic rankings
    q_emb = generic_model.encode(test['query'])
    doc_embs = generic_model.encode(test['docs'])
    gen_sims = [cosine_similarity([q_emb], [d])[0][0] for d in doc_embs]
    gen_ranking = np.argsort(gen_sims)[::-1]
    
    # Fine-tuned rankings
    q_emb = finetuned_model.encode(test['query'])
    doc_embs = finetuned_model.encode(test['docs'])
    ft_sims = [cosine_similarity([q_emb], [d])[0][0] for d in doc_embs]
    ft_ranking = np.argsort(ft_sims)[::-1]
    
    print("\n  Generic Model Ranking:")
    for rank, idx in enumerate(gen_ranking, 1):
        print(f"    {rank}. [{gen_sims[idx]:.3f}] {test['docs'][idx][:50]}...")
    
    print("\n  Fine-tuned Model Ranking:")
    for rank, idx in enumerate(ft_ranking, 1):
        print(f"    {rank}. [{ft_sims[idx]:.3f}] {test['docs'][idx][:50]}...")
    
    print()

## Summary

✅ Loaded 100 examples and created 200 pairs (positive + negative)  
✅ Used OnlineContrastiveLoss with 0.5 margin  
✅ Achieved better separation between similar and dissimilar  
✅ Improved classification accuracy  
✅ Better ranking of relevant documents  

**Key learnings:**
- Contrastive loss works with labeled pairs
- Simpler than triplet loss (no need for explicit negatives)
- Margin controls minimum distance for dissimilar pairs
- Works well with 200+ pairs (100 positive + 100 negative)
- Better separation in similarity distributions

**Contrastive vs Triplet:**
- **Contrastive**: Binary labels (similar/dissimilar), simpler data prep
- **Triplet**: Explicit triplets, more fine-grained control
- Both improve over generic models

**Next:** Learn full training with HuggingFace Transformers Trainer API!